# H02C8b Information Retrieval and Search Engines: RAG Project

Welcome to the notebook companion for the IRSE project. You will find all starter code here. You are encouraged to use this code, as it has been confirmed to work for the RAG pipeline described in the assignment handout. However, you are certainly welcome to make any changes you see fit, provided that your code is written in Python and runs without issue.

**IMPORTANT**: Do not submit a notebook as your final solution. It will not be graded. Refer to assignment handout for more information about the submission format.

**IMPORTANT**: Be mindful of your runtime usage, if working in Colab. At the beginning of every session, navigate to the top menu bar in Colab and select **Runtime > Change runtime type > CPU (Python 3)**. This will ensure that your session runs on CPU and that you do not waste any GPU allocation for the day. GPUs are provided by Google on a limited daily basis, and access is given every 24 hours. It is best that you complete the TF-IDF/search component before loading models and running inference on the GPU runtime.


If you have any questions, feel free to email [Thomas](mailto:thomas.bauwens@kuleuven.be) or [Kushal](mailto:kushaljayesh.tatariya@kuleuven.be).

## RAG for recipe recommendation:

We will begin by installing the huggingface `datasets` library for easily loading our data.

In [ ]:
! pip -q install datasets
!wget https://people.cs.kuleuven.be/~thomas.bauwens/irse_documents_2025_recipes.parquet
!wget https://people.cs.kuleuven.be/~thomas.bauwens/irse_queries_2025_recipes.json


In [1]:
import json
import nltk
import numpy as np
import pandas as pd
import json
import string
import datasets
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import math
import numpy as np
from nltk.corpus import stopwords

import string
from nltk.tokenize import word_tokenize
import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
tqdm.pandas()  # Show progress bar if using pandas

import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("punkt_tab")
nltk.download("wordnet")

# from google.colab import userdata
# userdata.get("HF_TOKEN")

/home/yan/yan_files/2_semester/information_retrieval/project/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/yan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/yan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/yan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/yan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/yan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/yan/nltk_data...
[nltk_data]   Package punkt_tab is al

True

In [2]:
dataset = datasets.load_dataset(
    "parquet", data_files="./irse_documents_2025_recipes.parquet"
)["train"]
queries_data = json.load(open("./irse_queries_2025_recipes.json", "r"))

df = dataset.to_pandas()

# Now you can apply the function to concatenate columns
recipies = df.apply(
    lambda row: f"{row['name']} {row['description']} {row['ingredients']} {row['steps']}", axis=1
)#[:10000]
recipe_ids = dataset["official_id"]#[:10000]
print("Number of documents:", len(recipies))

queries = pd.DataFrame(columns=["q", "r", "a"])
for query_item in queries_data["queries"]:
    query_text = query_item["q"]
    relevance_pairs = query_item["r"]
    answer = query_item["a"]
    queries = pd.concat(
        [
            queries,
            pd.DataFrame({"q": [query_text], "r": [relevance_pairs], "a": [answer]}),
        ],
        ignore_index=True,
    )

print("Number of queries:", len(queries))

Number of documents: 231637
Number of queries: 47


In [ ]:


stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_fast(doc):
    doc = doc.translate(str.maketrans("", "", string.punctuation)).lower()

    words = word_tokenize(doc)

    words = [
        lemmatizer.lemmatize(word)
        for word in words
        if word not in stop_words #and word.isalpha()
    ]

    return " ".join(words)

preprocessed_recipes = [preprocess_fast(doc) for doc in tqdm(recipies)]



100%|██████████| 231637/231637 [02:03<00:00, 1879.62it/s]


#### Preprocessor

In [ ]:
# lemmatizer = WordNetLemmatizer()
# counter = 0


# def preprocess(doc):
#     global counter
#     counter += 1
#     if counter % 1000 == 0:
#         print(f"Processed: {counter}")
#     doc = doc.split()
#     preprocessed_text = []
#     for text in doc:
#         text = text.translate(str.maketrans("", "", string.punctuation))
#         text = text.lower()
#         words = word_tokenize(text)
#         words = [word for word in words if word not in stopwords.words("english")]
#         words = [lemmatizer.lemmatize(word) for word in words]
#         if words != []:
#             preprocessed_text.append(words[0])
#     return preprocessed_text
# vectorizer = TfidfVectorizer(tokenizer=preprocess)
# X = vectorizer.fit_transform(recipies)
# print("fitted")
# tfidf_df = pd.DataFrame(
#     X.toarray(), index=range(len(recipies)), columns=vectorizer.get_feature_names_out()
# )


In [6]:
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(preprocessed_recipes)
print("TF-IDF fitted")

# # Optional: convert to DataFrame (can be memory-heavy)
# tfidf_df = pd.DataFrame(
#     X.toarray(), index=range(len(preprocessed_recipes)), columns=vectorizer.get_feature_names_out()
# )


TF-IDF fitted


#### TF-IDF

In [9]:
def retrieve_documents(query_text, k=5):
    query = preprocess_fast(query_text)
    query_vector = vectorizer.transform([" ".join(query)])
    cosine_similarities = cosine_similarity(query_vector, X)
    results = [
        (recipies[i], recipe_ids[i], cosine_similarities[0][i])
        for i in range(len(recipies))
    ]
    results.sort(key=lambda x: x[2], reverse=True)
    for doc, id, similarity in results[:k]:
        print(f"Similarity: {similarity:.2f}\n{id}:{doc}\n")
    return results[:k]


def calculate_precision_recall_f1_optimized(relevant_doc_ids, retrieved_doc_ids):
    # Convert to sets for faster operations
    relevant_set = set(relevant_doc_ids)
    retrieved_set = set(retrieved_doc_ids)

    # Calculate true positives (documents that are both relevant and retrieved)
    true_positives = len(relevant_set.intersection(retrieved_set))

    # Calculate precision, recall, and F1
    if len(retrieved_set) == 0:
        precision = 0.0
        recall = 0.0 if len(relevant_set) > 0 else 1.0
        f1 = 0.0
    elif len(relevant_set) == 0:
        precision = 0.0
        recall = 1.0
        f1 = 0.0
    else:
        precision = true_positives / len(retrieved_set)
        recall = true_positives / len(relevant_set)
        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0.0

    return {"precision": precision, "recall": recall, "f1": f1}


def calculate_macro_averages(metrics_per_query):
    # This function remains the same
    precision_values = [metrics["precision"] for metrics in metrics_per_query]
    recall_values = [metrics["recall"] for metrics in metrics_per_query]
    f1_values = [metrics["f1"] for metrics in metrics_per_query]

    macro_precision = np.mean(precision_values)
    macro_recall = np.mean(recall_values)
    macro_f1 = np.mean(f1_values)

    return {
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
    }


def calculate_micro_averages_optimized(all_relevant_doc_ids, all_retrieved_doc_ids):
    # Flatten lists of relevant and retrieved document IDs
    all_relevant = [
        doc_id for query_relevant in all_relevant_doc_ids for doc_id in query_relevant
    ]
    all_retrieved = [
        doc_id
        for query_retrieved in all_retrieved_doc_ids
        for doc_id in query_retrieved
    ]

    # Count true positives across all queries
    relevant_set = set(all_relevant)
    retrieved_set = set(all_retrieved)
    true_positives = len(relevant_set.intersection(retrieved_set))

    # Calculate micro-averaged metrics
    if len(retrieved_set) == 0:
        micro_precision = 0.0
        micro_recall = 0.0 if len(relevant_set) > 0 else 1.0
        micro_f1 = 0.0
    elif len(relevant_set) == 0:
        micro_precision = 0.0
        micro_recall = 1.0
        micro_f1 = 0.0
    else:
        micro_precision = true_positives / len(retrieved_set)
        micro_recall = true_positives / len(relevant_set)
        if micro_precision + micro_recall > 0:
            micro_f1 = (
                2 * micro_precision * micro_recall / (micro_precision + micro_recall)
            )
        else:
            micro_f1 = 0.0

    return {
        "micro_precision": micro_precision,
        "micro_recall": micro_recall,
        "micro_f1": micro_f1,
    }

In [10]:
def evaluate_ir_system(queries, recipies, recipe_ids, k=5):
    metrics_per_query = []
    all_relevant_doc_ids = []
    all_retrieved_doc_ids = []
    

    for i in range(len(queries)):
        query_text = queries.q[i]
        relevant_doc_ids = queries.r[i]
        relevant_doc_ids = [doc[0] for doc in relevant_doc_ids]

        print(f"\nProcessing query {i + 1}/{len(queries)}: {query_text}")

        results = retrieve_documents(query_text, k)
        retrieved_doc_ids = [result[1] for result in results]

        # Calculate metrics directly using the optimized function
        query_metrics = calculate_precision_recall_f1_optimized(relevant_doc_ids, retrieved_doc_ids)
        metrics_per_query.append(query_metrics)

        # Store for micro-averaging
        all_relevant_doc_ids.append(relevant_doc_ids)
        all_retrieved_doc_ids.append(retrieved_doc_ids)

        print(
            f"Query {i + 1} metrics: Precision={query_metrics['precision']:.4f}, "
            f"Recall={query_metrics['recall']:.4f}, F1={query_metrics['f1']:.4f}"
        )

    # Calculate macro and micro averages
    macro_metrics = calculate_macro_averages(metrics_per_query)
    micro_metrics = calculate_micro_averages_optimized(all_relevant_doc_ids, all_retrieved_doc_ids)

    # Combine all metrics
    all_metrics = {**macro_metrics, **micro_metrics}

    return all_metrics

In [11]:
retrieve_documents("What temperature should I pre-heat my oven to when making chicken quesadillas?")

Similarity: 0.00
1:arriba baked winter squash mexican style autumn is my favorite time of year to cook! this recipe 
can be prepared either spicy or sweet, your choice!
two of my posted mexican-inspired seasoning mix recipes are offered as suggestions. winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt make a choice and proceed with recipe, depending on size of squash , cut into half or fourths, remove seeds, for spicy squash , drizzle olive oil or melted butter over each cut squash piece, season with mexican seasoning mix ii, for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece, season with sweet mexican spice mix, bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin, be careful not to burn the squash especially if you opt to use sugar or butter, if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of

[('arriba baked winter squash mexican style autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions. winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt make a choice and proceed with recipe, depending on size of squash , cut into half or fourths, remove seeds, for spicy squash , drizzle olive oil or melted butter over each cut squash piece, season with mexican seasoning mix ii, for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece, season with sweet mexican spice mix, bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin, be careful not to burn the squash especially if you opt to use sugar or butter, if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking, i

In [12]:

# Evaluate the IR system
print("\nEvaluating IR system...")
metrics = evaluate_ir_system(queries, recipies, recipe_ids, k=5)

# Print the results
print("\n===== IR System Evaluation Results =====")
print(f"Macro-average Precision: {metrics['macro_precision']:.4f}")
print(f"Macro-average Recall: {metrics['macro_recall']:.4f}")
print(f"Macro-average F1: {metrics['macro_f1']:.4f}")
print(f"Micro-average Precision: {metrics['micro_precision']:.4f}")
print(f"Micro-average Recall: {metrics['micro_recall']:.4f}")
print(f"Micro-average F1: {metrics['micro_f1']:.4f}")
print("========================================")


Evaluating IR system...

Processing query 1/47: What temperature should I pre-heat my oven to when making chicken quesadillas?
Similarity: 0.00
1:arriba baked winter squash mexican style autumn is my favorite time of year to cook! this recipe 
can be prepared either spicy or sweet, your choice!
two of my posted mexican-inspired seasoning mix recipes are offered as suggestions. winter squash, mexican seasoning, mixed spice, honey, butter, olive oil, salt make a choice and proceed with recipe, depending on size of squash , cut into half or fourths, remove seeds, for spicy squash , drizzle olive oil or melted butter over each cut squash piece, season with mexican seasoning mix ii, for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece, season with sweet mexican spice mix, bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin, be careful not to burn the squash especially if you opt to

For a given query and set of relevant documents, you are also required to create a prompt that instructs a model to complete a certain task (e.g. recipe recommendation). You should experiment with formatting the prompt, as language models have been shown to be sensitive to the exact verbiage of instructions.

In [ ]:
prompt = """

# Recipe Assistant

## Context
You are a helpful recipe assistant with access to a database of recipes. The system has already retrieved the most relevant recipes to the user's query using TF-IDF similarity. Your goal is to provide helpful, accurate responses about recipes, cooking techniques, ingredient substitutions, and culinary advice based on the retrieved recipes.

## Retrieved Recipes
The following recipes have been retrieved as most relevant to the user's query:

{retrieved_recipes}

## Instructions
1. **Answer directly from the retrieved recipes when possible.** Use the information from the provided recipes to answer questions about ingredients, cooking methods, nutritional information, and preparation steps.

2. **For ingredient questions:**
   - Provide accurate amounts and measurements from the recipes
   - Suggest possible substitutions based on common culinary knowledge
   - Explain the purpose of key ingredients in the dish

3. **For cooking technique questions:**
   - Explain preparation methods mentioned in the recipes
   - Clarify cooking times and temperatures
   - Describe expected results and how to tell when food is properly cooked

4. **For modification requests:**
   - Suggest appropriate adjustments for dietary restrictions (vegan, gluten-free, etc.)
   - Explain how to scale recipes up or down
   - Offer ideas for flavor variations while maintaining the core identity of the dish

5. **For general questions:**
   - Provide brief culinary background/history when relevant
   - Explain unfamiliar cooking terms
   - Suggest pairings, serving suggestions, and storage recommendations

## Response Format
- Start with a direct answer to the user's question
- Keep your responses concise but comprehensive
- For multi-step instructions or complex concepts, organize information in a clear, logical structure
- If the retrieved recipes don't contain sufficient information to answer the query, acknowledge the limitations and provide general culinary knowledge that might help
- When suggesting modifications not explicitly in the retrieved recipes, clearly indicate these are your recommendations based on culinary principles

## Limitations
- Don't make claims about specific nutritional values unless they're mentioned in the retrieved recipes
- If asked about topics completely unrelated to cooking or the recipes provided, politely redirect the conversation back to recipe-related topics
- Don't invent or fabricate details about recipes that aren't in the retrieved data

## User Query
{user_query}
"""

In [ ]:
irrelevant_context = """
Richard Gary Brautigan (January 30, 1935 – c. September 16, 1984)
was an American novelist, poet, and short story writer. A prolific writer,
he wrote throughout his life and published ten novels, two collections of
short stories, and four books of poetry. Brautigan's work has been published
both in the United States and internationally throughout Europe, Japan,
and China. He is best known for his novels Trout Fishing in America (1967),
In Watermelon Sugar (1968), and The Abortion: An Historical Romance 1966 (1971).
"""

**IMPORTANT**: only run the following code when you have implemented a working retrieval system. When you are ready to work with language models, navigate to the menu bar in Colab and select **Runtime > Change runtime type > T4 GPU**. If you find yourself working on not GPU-intenstive tasks in this notebook, change your runtime back to CPU to preserve access.


In [ ]:
! pip -q install git+https://github.com/huggingface/transformers
! pip -q install datasets bitsandbytes accelerate xformers einops

In [ ]:
import torch
import transformers
import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata


In [ ]:
from huggingface_hub import login

# Replace "YOUR_HF_TOKEN" with your actual Hugging Face token
login(token=userdata.get('HF_TOKEN'))

The code below will load a Mistral 7B instruct model and quantize it via `bitesandbytes`. Doing so will ensure that the model will not take up too much memory and make inference more efficient. Note that the call to `AutoModelForCausalLM.from_pretrained()` will take a while, as the model's weights must be downloaded from the huggingface hub. Also note that you are not restricted to using Mistral, and are welcome to experiment with other models (though you will have more luck with chat and instruction-tuned variants).

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, quantization_config=bnb_config, device_map="auto"
)

A tokenizer is required in order to convert strings into integer sequences that can be passed as input to the model.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
retrieved_recipes = "1. Chocolate Chip Cookies...\n2. Brownie Bites..."
user_query = "Can I use coconut oil instead of butter in cookies?"

# Fill in the template
input_string_with_context = prompt.format(
    retrieved_recipes=retrieved_recipes, user_query=user_query
)

input_string_without_context = prompt.format(
    retrieved_recipes=irrelevant_context, user_query=user_query
)

In [ ]:
encoded_prompt = tokenizer(
    input_string_with_context, return_tensors="pt", add_special_tokens=False
)
encoded_prompt = encoded_prompt.to("cuda")
generated_ids = model.generate(**encoded_prompt, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [ ]:
encoded_prompt = tokenizer(
    input_string_without_context, return_tensors="pt", add_special_tokens=False
)
encoded_prompt = encoded_prompt.to("cuda")
generated_ids = model.generate(**encoded_prompt, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])